In [1]:
import xarray as xr
import numpy as np
import dask
import matplotlib.pyplot as plt
from utils import geo
import pandas as pd
import regionmask
from dask.diagnostics import ProgressBar
import glob
import os

In [7]:
dataset = 'oc-cci'
frequency = '8day'
variable = 'chlor_a'
yearstart = 1999
yearend = 2023
monthorder = 'JULtoJUN'
averaging = 'wmean-latlon'

In [17]:
if dataset == 'globcolour-hermes':
    rootdir = '/projects/SOCCOM/datasets/GlobColour'
    local = 'HERMES/merged/'+frequency+'/*/*/*'
    mode = 'AV'
    filename = '*'+mode+'*'+variable+'*.nc'
    filetype = 'netcdf'
    
    def preprocess(ds):
        time = pd.to_datetime(ds.attrs['start_time'])
        if 'CHL1_error' in ds.data_vars:
            ds = ds.drop(['CHL1_error'])
        ds = ds.sel({'lat':slice(-30,-65)}).expand_dims({'time':[time]}).rename({'CHL1_mean':variable})
        return ds
    
elif dataset == 'globcolour-cmems':
    rootdir = '/projects/SOCCOM/datasets/GlobColour'
    local = 'CMEMS/OCEANCOLOUR_GLO_BGC_L4_MY_009_104-TDS/SouthernOcean_30Sto65S/'+frequency+'/bymonth.cmc'
    filename = 'cmems_obs-oc_glo_bgc-plankton_my_l4-gapfree-multi-4km_P1D_*.nc'
    filetype = 'netcdf'
    
    def preprocess(ds):
        time = pd.to_datetime(ds.attrs['start_time'])
        # This is a hack because of issues with data download
        ds = ds.where(ds['time.month']==ds['time.month'][0],drop=True).rename({'CHL':variable,'longitude':'lon','latitude':'lat'})
        return ds.copy()
    
elif dataset == 'oc-cci':
    rootdir = "/projects/SOCCOM/datasets/OC-CCI/"
    lats = [-30, -65]
    latstr = "_".join(["lat",*[str(lat) for lat in lats]])
    dataname = "CCI_ALL-v6.0-8DAY"
    local = "/".join([dataname,latstr])
    filename = dataname+"-"+latstr+"-gapfilled_lonlattime-roll_time3"
    filetype = 'zarr'
    
print('Opening.')
path = '/'.join([rootdir,local,filename])
with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    if filetype=='nc':
        ds = xr.open_mfdataset(path,preprocess=preprocess,use_cftime=False)
    elif filetype=='zarr':
        ds = xr.open_zarr(path).chunk({'time':'auto','lon':-1,'lat':-1})

ds['time'] = pd.to_datetime(ds['time'].values)
# Get area
ds,xgrid = geo.get_xgcm_horizontal(ds,axes_dims_dict={'X':'lon','Y':'lat'},periodic='X')
ds['area'] = ds['dxC']*ds['dyC']

Opening.


In [18]:
ds = ds.chunk({'time':'auto'})
ds

<xarray.Dataset>
Dimensions:   (time: 1212, lat: 841, lon: 8640, lon_left: 8640, lat_left: 841)
Coordinates:
  * lat       (lat) float64 -30.02 -30.06 -30.1 -30.15 ... -64.94 -64.98 -65.02
  * lon       (lon) float64 -180.0 -179.9 -179.9 -179.9 ... 179.9 179.9 180.0
  * time      (time) datetime64[ns] 1997-09-04 1997-09-06 ... 2023-12-27
  * lon_left  (lon_left) float64 -180.0 -180.0 -179.9 ... 179.9 179.9 180.0
  * lat_left  (lat_left) float64 -30.0 -30.04 -30.08 ... -64.92 -64.96 -65.0
Data variables:
    chlor_a   (time, lat, lon) float32 dask.array<chunksize=(4, 841, 8640), meta=np.ndarray>
    dxG       (lon_left, lat) float64 dask.array<chunksize=(8640, 841), meta=np.ndarray>
    dyG       (lon, lat_left) float64 dask.array<chunksize=(8640, 841), meta=np.ndarray>
    dxC       (lon, lat) float64 dask.array<chunksize=(8640, 841), meta=np.ndarray>
    dyC       (lon, lat) float64 dask.array<chunksize=(8640, 841), meta=np.ndarray>
    rC        (lon, lat) float64 dask.array<chunksize=(8640, 841), meta=np.ndarray>
    area      (lon, lat) float64 dask.array<chunksize=(8640, 841), meta=np.ndarray>
Attributes: (12/50)
    Conventions:                       CF-1.7
    Metadata_Conventions:              Unidata Dataset Discovery v1.0
    NCO:                               netCDF Operators version 4.7.5 (Homepa...
    cdm_data_type:                     Grid
    comment:                           See summary attribute
    creation_date:                     Thu Jan 18 08:36:14 2024
    ...                                ...
    time_coverage_duration:            P8D
    time_coverage_end:                 202312262359Z
    time_coverage_resolution:          P8D
    time_coverage_start:               202312190000Z
    title:                             ESA CCI Ocean Colour Product
    tracking_id:                       314b5bdb-9adb-4896-890e-441d9f9aecd5

In [19]:
# Apply a landmask
mask = regionmask.defined_regions.natural_earth.land_110.mask(ds['lon'], ds['lat'])
mask = mask.where(np.isfinite(mask),1)
mask = mask.where(mask==1,np.nan)
# mask.plot()
ds[variable] = ds[variable].fillna(0)*mask

/home/graemem/.conda/envs/core/lib/python3.10/site-packages/regionmask/defined_regions/_natural_earth.py:471: FutureWarning: ``regionmask.defined_regions.natural_earth`` is deprecated. Please use ``regionmask.defined_regions.natural_earth_v4_1_0`` or ``regionmask.defined_regions.natural_earth_v5_0_0`` instead.
  warnings.warn(


In [20]:
# Extract spatial average for each year
years = np.arange(yearstart,yearend+1)
days = np.arange(0,365)
# Ordering such that years run from July to June
ds_year = xr.DataArray(dims=['day','year'],coords={'year':years,'day':days},name=variable)
for i,year in enumerate(years):
    print(year)
    if monthorder=='JULtoJUN':
        start = str(year-1)+'-07-01'
        end = str(year)+'-06-30'
        nleapday = 243 # for removing leap day (243 days after July 1)
    elif monthorder=='JANtoDEC':
        start = str(year-1)+'-01-01'
        end = str(year)+'-12-31'
        nleapday = 59 # for removing leap day (59 days after Jan 1)
    
    # Weighted spatial mean
    x = (ds[variable].sel({'time':slice(start,end)})).weighted(ds['area'].fillna(0)).mean(['lat','lon'])
    
    # Interpolate to daily
    # (day frquency data has some missing days, so interpolate that too)
    alltimes = pd.date_range(start,end)
    xi = x.interp({'time':alltimes},kwargs={'fill_value':np.nan})
        
    # Remove leap-year day for simplicity
    if len(xi)==366:
        xi = xi[np.arange(len(xi))!=nleapday]
        
    # Put into dataset
    with ProgressBar():
        xi = xi.compute()
    ds_year.loc[{'year':year}]=xi.values

1999
[########################################] | 100% Completed | 4.03 sms
2000
[########################################] | 100% Completed | 2.67 sms
2001
[########################################] | 100% Completed | 3.30 sms
2002
[########################################] | 100% Completed | 2.96 sms
2003
[########################################] | 100% Completed | 4.31 sms
2004
[########################################] | 100% Completed | 4.08 sms
2005
[########################################] | 100% Completed | 3.00 sms
2006
[########################################] | 100% Completed | 3.96 sms
2007
[########################################] | 100% Completed | 3.09 sms
2008
[########################################] | 100% Completed | 4.09 sms
2009
[########################################] | 100% Completed | 4.18 sms
2010
[########################################] | 100% Completed | 3.05 sms
2011
[########################################] | 100% Completed | 3.98 sms
2012
[######

In [21]:
outdir = '../../data/'
yearstr = str(yearstart)+'-'+str(yearend)
outpath = outdir+'.'.join([dataset,frequency,averaging,monthorder,yearstr,'nc'])

In [22]:
print('Saving to '+outpath)
if os.path.isfile(outpath):
    os.remove(outpath)
ds_year.to_netcdf(outpath)

Saving to ../../data/oc-cci.8day.wmean-latlon.JULtoJUN.1999-2023.nc


### By basin

In [23]:
atlind = 20
indwpac = 170
wpacepac = -130
epacatl = -70

cond_atlantic = (ds['lon']>=epacatl) & (ds['lon']<atlind)
cond_indian = (ds['lon']>=atlind) & (ds['lon']<indwpac)
cond_wpacific = (ds['lon']>=indwpac) | (ds['lon']<wpacepac)
cond_epacific = (ds['lon']>=wpacepac) & (ds['lon']<epacatl)

atlantic = ds[variable].where(cond_atlantic,drop=True)
atlantic_area = ds['area'].where(cond_atlantic,drop=True)
indian = ds[variable].where(cond_indian,drop=True)
indian_area = ds['area'].where(cond_indian,drop=True)
wpacific = ds[variable].where(cond_wpacific,drop=True)
wpacific_area = ds['area'].where(cond_wpacific,drop=True)
epacific = ds[variable].where(cond_epacific,drop=True)
epacific_area = ds['area'].where(cond_epacific,drop=True)

/home/graemem/.conda/envs/core/lib/python3.10/site-packages/xarray/core/indexing.py:1446: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]
/home/graemem/.conda/envs/core/lib/python3.10/site-packages/xarray/core/indexing.py:1446: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]
/home/

In [24]:
years = np.arange(yearstart,yearend+1)
days = np.arange(0,365)
# Extract spatial average for each year
atlantic_year = xr.DataArray(dims=['day','year'],coords={'year':years,'day':days},name=variable)
indian_year = xr.DataArray(dims=['day','year'],coords={'year':years,'day':days},name=variable)
wpacific_year = xr.DataArray(dims=['day','year'],coords={'year':years,'day':days},name=variable)
epacific_year = xr.DataArray(dims=['day','year'],coords={'year':years,'day':days},name=variable)
for i,year in enumerate(years):
    print(year)
    start = str(year-1)+'-07-01'
    end = str(year)+'-06-30'
    alltimes = pd.date_range(start,end)
    
    # Atlantic
    print('atlantic')
    x = (atlantic.sel({'time':slice(start,end)})).weighted(atlantic_area.fillna(0)).mean(['lat','lon'])
    x = x.interp({'time':alltimes})
    if len(x)==366:
        # Just take out the leap year day
        # (Always 243 days after July 1)
        x = x[np.arange(len(x))!=243]
    # Put into dataset
    with ProgressBar():
        atlantic_year.loc[{'year':year}]=x.values
        
    # Indian
    print('indian')
    x = (indian.sel({'time':slice(start,end)})).weighted(indian_area.fillna(0)).mean(['lat','lon'])
    x = x.interp({'time':alltimes})
    if len(x)==366:
        # Just take out the leap year day
        # (Always 243 days after July 1)
        x = x[np.arange(len(x))!=243]
    # Put into dataset
    with ProgressBar():
        indian_year.loc[{'year':year}]=x.values
        
    # West Pacific
    print('wpacific')
    x = (wpacific.sel({'time':slice(start,end)})).weighted(wpacific_area.fillna(0)).mean(['lat','lon'])
    x = x.interp({'time':alltimes})
    if len(x)==366:
        # Just take out the leap year day
        # (Always 243 days after July 1)
        x = x[np.arange(len(x))!=243]
    # Put into dataset
    with ProgressBar():
        wpacific_year.loc[{'year':year}]=x.values
        
    # East Pacific
    print('epacific')
    x = (epacific.sel({'time':slice(start,end)})).weighted(epacific_area.fillna(0)).mean(['lat','lon'])
    x = x.interp({'time':alltimes})
    if len(x)==366:
        # Just take out the leap year day
        # (Always 243 days after July 1)
        x = x[np.arange(len(x))!=243]
    # Put into dataset
    with ProgressBar():
        epacific_year.loc[{'year':year}]=x.values

1999
atlantic
[########################################] | 100% Completed | 3.63 sms
indian
[########################################] | 100% Completed | 4.09 sms
wpacific
[########################################] | 100% Completed | 3.58 sms
epacific
[########################################] | 100% Completed | 3.35 sms
2000
atlantic
[########################################] | 100% Completed | 2.33 sms
indian
[########################################] | 100% Completed | 2.54 sms
wpacific
[########################################] | 100% Completed | 2.23 sms
epacific
[########################################] | 100% Completed | 2.14 sms
2001
atlantic
[########################################] | 100% Completed | 3.19 sms
indian
[########################################] | 100% Completed | 3.31 sms
wpacific
[########################################] | 100% Completed | 2.81 sms
epacific
[########################################] | 100% Completed | 2.98 sms
2002
atlantic
[################

In [25]:
outdir = '../../data/'
yearstr = str(yearstart)+'-'+str(yearend)

for basin in ['atlantic','indian','wpacific','epacific']:
    outpath = outdir+'.'.join([dataset,frequency,averaging,monthorder,yearstr,basin,'nc'])
    if basin=='atlantic':
        atlantic_year.to_netcdf(outpath)
    elif basin=='indian':
        indian_year.to_netcdf(outpath)
    elif basin=='wpacific':
        wpacific_year.to_netcdf(outpath)
    elif basin=='epacific':
        epacific_year.to_netcdf(outpath)